<a href="https://colab.research.google.com/github/bkpearse/Cooking-Project/blob/main/Automate_FPL_spreadheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, json
from pprint import pprint
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

In [ ]:
!pip install aiohttp
!pip install understat

In [ ]:
import aiohttp
import asyncio
from understat import Understat

###Import the file containing the API Key


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fpl-spreadsheet.json to fpl-spreadsheet.json


##Give access to the API key


In [ ]:
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']

try:
    creds = ServiceAccountCredentials.from_json_keyfile_name('fpl-spreadsheet.json', scope)  # The file is now just called 'fpl-spreadsheet.json' without the 'C:/FPL API/' prefix
    client = gspread.authorize(creds)
    print("Success")
except Exception as e:
    print(f"An error occurred: {e}")

Success


In [ ]:
# Open the Google Sheets
sheet = client.open("FPL Spreadsheet")

In [ ]:
# Get the "Player Data" worksheet
player_data_sheet = sheet.worksheet("Player Data")
team_data_sheet = sheet.worksheet("Team Data")
fixtures_data_sheet = sheet.worksheet("Upcoming Fixtures")

In [ ]:
# Set up the API endpoint and make the request
base_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(base_url)

In [ ]:

# Step 3: Make the API request to the Premier League API and create a DataFrame with the data
response = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
data = response.json()
players_data = data['elements']
df = pd.DataFrame(players_data)

In [ ]:
column_data = df['second_name'].to_list()
column_data_for_gsheets = [[item] for item in column_data]
player_data_sheet.update('A2', column_data_for_gsheets, value_input_option='RAW')

In [ ]:
column_data = df['expected_goals'].to_list()
column_data_for_gsheets = [[item] for item in column_data]
player_data_sheet.update('B2', column_data_for_gsheets, value_input_option='RAW')


In [ ]:
column_data = df['expected_assists'].to_list()
column_data_for_gsheets = [[item] for item in column_data]
player_data_sheet.update('C2', column_data_for_gsheets, value_input_option='RAW')

In [ ]:
column_data = df['team'].to_list()
column_data_for_gsheets = [[item + 100] for item in column_data]
player_data_sheet.update('AK2', column_data_for_gsheets, value_input_option='RAW')

In [ ]:
column_data = df['minutes'].to_list()
column_data_for_gsheets = [[item] for item in column_data]
player_data_sheet.update('J2', column_data_for_gsheets, value_input_option='RAW')

In [ ]:
column_data = df['goals_scored'].to_list()
column_data_for_gsheets = [[item] for item in column_data]
player_data_sheet.update('N2', column_data_for_gsheets, value_input_option='RAW')

In [ ]:
column_data = df['assists'].to_list()
column_data_for_gsheets = [[item] for item in column_data]
player_data_sheet.update('O2', column_data_for_gsheets, value_input_option='RAW')

#UNDERSTAT DATA COLLECTION


In [ ]:
import nest_asyncio


In [ ]:

nest_asyncio.apply()

### Update Player Information

In [ ]:
async def main():
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        player = await understat.get_league_players("epl", 2023)
        player_names = [x['player_name'] for x in player]
        print(f'There are {len(player)} players in the list.')
        print(json.dumps(player))
        # Extract the data into lists
        player_names = [x['player_name'] for x in player]
        games_played = [x['games'] for x in player]
        mins_played = [x['time'] for x in player]
        goals_scored = [x['goals'] for x in player]
        expected_goals = [x['xG'] for x in player]
        assists_provided = [x['assists'] for x in player]
        expected_assists = [x['xA'] for x in player]
        position_played = [x['position'] for x in player]
        team_name = [x['team_title'] for x in player]
        shots_taken = [x['shots'] for x in player]
        passes_taken = [x['key_passes'] for x in player]


        # Update Google Sheet
        player_data_sheet.update('A2', [[name] for name in player_names])
        player_data_sheet.update('K2', [[int(game)] for game in games_played])
        player_data_sheet.update('J2', [[int(x)] for x in mins_played])
        player_data_sheet.update('N2', [[int(x)] for x in goals_scored])
        player_data_sheet.update('B2', [[float(x)] for x in expected_goals])
        player_data_sheet.update('O2', [[int(x)] for x in assists_provided])
        player_data_sheet.update('C2', [[float(x)] for x in expected_assists])
        player_data_sheet.update('I2', [[name] for name in position_played])
        player_data_sheet.update('H2', [[name] for name in team_name])
        player_data_sheet.update('E2', [[int(x)] for x in shots_taken])
        player_data_sheet.update('F2', [[int(x)] for x in passes_taken])

asyncio.run(main())

There are 405 players in the list.
[{"id": "8260", "player_name": "Erling Haaland", "games": "5", "time": "445", "goals": "7", "xG": "7.646338880062103", "assists": "1", "xA": "1.1254648137837648", "shots": "29", "key_passes": "7", "yellow_cards": "0", "red_cards": "0", "position": "F", "team_title": "Manchester City", "npg": "6", "npxG": "6.124001145362854", "xGChain": "6.56243109703064", "xGBuildup": "0.5850657597184181"}, {"id": "3697", "player_name": "Odsonne Edouard", "games": "5", "time": "432", "goals": "4", "xG": "3.337449833750725", "assists": "0", "xA": "0.26749951019883156", "shots": "20", "key_passes": "5", "yellow_cards": "0", "red_cards": "0", "position": "F", "team_title": "Crystal Palace", "npg": "4", "npxG": "3.337449833750725", "xGChain": "3.16320563852787", "xGBuildup": "0.4057522751390934"}, {"id": "6552", "player_name": "Bryan Mbeumo", "games": "5", "time": "450", "goals": "4", "xG": "3.8939481377601624", "assists": "0", "xA": "0.47218552231788635", "shots": "10", 

### Update Team Information

In [ ]:
async def main():
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        league_table = await understat.get_league_table("epl", 2023)
        keys = league_table[0]
        values = league_table[1:]
        teams = [dict(zip(keys,value)) for value in values]
        print(teams)
        # Extract the data into lists
        team_names = [x['Team'] for x in teams]
        games_played = [x['M'] for x in teams]
        expected_goals = [x['xG'] for x in teams]
        expected_against = [x['xGA'] for x in teams]



        # Update Google Sheet
        team_data_sheet.update('A2', [[name] for name in team_names])
        team_data_sheet.update('F2', [[int(x)] for x in games_played])
        team_data_sheet.update('B2', [[float(x)] for x in expected_goals])
        team_data_sheet.update('C2', [[float(x)] for x in expected_against])

asyncio.run(main())

[{'Team': 'Manchester City', 'M': 5, 'W': 5, 'D': 0, 'L': 0, 'G': 14, 'GA': 3, 'PTS': 15, 'xG': 13.63, 'NPxG': 12.1, 'xGA': 3.69, 'NPxGA': 3.69, 'NPxGD': 8.41, 'PPDA': 10.88, 'OPPDA': 26.45, 'DC': 64, 'ODC': 17, 'xPTS': 12.65}, {'Team': 'Tottenham', 'M': 5, 'W': 4, 'D': 1, 'L': 0, 'G': 13, 'GA': 5, 'PTS': 13, 'xG': 10.55, 'NPxG': 10.55, 'xGA': 7.26, 'NPxGA': 6.5, 'NPxGD': 4.05, 'PPDA': 8.21, 'OPPDA': 11.77, 'DC': 74, 'ODC': 40, 'xPTS': 9.16}, {'Team': 'Liverpool', 'M': 5, 'W': 4, 'D': 1, 'L': 0, 'G': 12, 'GA': 4, 'PTS': 13, 'xG': 10.97, 'NPxG': 10.05, 'xGA': 6.09, 'NPxGA': 6.09, 'NPxGD': 3.95, 'PPDA': 12.1, 'OPPDA': 16.67, 'DC': 52, 'ODC': 36, 'xPTS': 9.76}, {'Team': 'Brighton', 'M': 5, 'W': 4, 'D': 0, 'L': 1, 'G': 15, 'GA': 7, 'PTS': 12, 'xG': 12.16, 'NPxG': 11.4, 'xGA': 9.15, 'NPxGA': 8.39, 'NPxGD': 3.01, 'PPDA': 8.39, 'OPPDA': 18.02, 'DC': 67, 'ODC': 30, 'xPTS': 8.88}, {'Team': 'Arsenal', 'M': 4, 'W': 3, 'D': 1, 'L': 0, 'G': 8, 'GA': 4, 'PTS': 10, 'xG': 8.19, 'NPxG': 6.67, 'xGA': 3.

### Update Fixtures


In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B14', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Manchester City"))

[{'id': '21946', 'isResult': False, 'side': 'h', 'h': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'a': {'id': '249', 'title': 'Nottingham Forest', 'short_title': 'NOT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21961', 'isResult': False, 'side': 'a', 'h': {'id': '229', 'title': 'Wolverhampton Wanderers', 'short_title': 'WOL'}, 'a': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21965', 'isResult': False, 'side': 'a', 'h': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'a': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21980', 'isResult': False, 'side': 'h', 'h': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'a': {'id': '220', 'tit

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B2', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Arsenal"))

[{'id': '21941', 'isResult': False, 'side': 'a', 'h': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'a': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-17 15:30:00'}, {'id': '21952', 'isResult': False, 'side': 'h', 'h': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'a': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 13:00:00'}, {'id': '21959', 'isResult': False, 'side': 'a', 'h': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'a': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21965', 'isResult': False, 'side': 'h', 'h': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'a': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'goals':

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B3', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Aston Villa"))

[{'id': '21947', 'isResult': False, 'side': 'a', 'h': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'}, 'a': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21955', 'isResult': False, 'side': 'h', 'h': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'a': {'id': '220', 'title': 'Brighton', 'short_title': 'BRI'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 11:30:00'}, {'id': '21974', 'isResult': False, 'side': 'a', 'h': {'id': '229', 'title': 'Wolverhampton Wanderers', 'short_title': 'WOL'}, 'a': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21976', 'isResult': False, 'side': 'h', 'h': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'a': {'id': '81', 'title': 'West Ham', 'short_ti

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B4', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Bournemouth"))

[{'id': '21942', 'isResult': False, 'side': 'h', 'h': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'a': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-17 13:00:00'}, {'id': '21951', 'isResult': False, 'side': 'a', 'h': {'id': '220', 'title': 'Brighton', 'short_title': 'BRI'}, 'a': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 13:00:00'}, {'id': '21959', 'isResult': False, 'side': 'h', 'h': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'a': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21969', 'isResult': False, 'side': 'a', 'h': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'a': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'goa

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B5', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Brentford"))

[{'id': '21949', 'isResult': False, 'side': 'h', 'h': {'id': '244', 'title': 'Brentford', 'short_title': 'BRE'}, 'a': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 16:30:00'}, {'id': '21963', 'isResult': False, 'side': 'a', 'h': {'id': '249', 'title': 'Nottingham Forest', 'short_title': 'NOT'}, 'a': {'id': '244', 'title': 'Brentford', 'short_title': 'BRE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-01 13:00:00'}, {'id': '21972', 'isResult': False, 'side': 'a', 'h': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'}, 'a': {'id': '244', 'title': 'Brentford', 'short_title': 'BRE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21977', 'isResult': False, 'side': 'h', 'h': {'id': '244', 'title': 'Brentford', 'short_title': 'BRE'}, 'a': {'id': '92', 'title': 'Burnley', 'short_title

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B6', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Brighton"))

[{'id': '21951', 'isResult': False, 'side': 'h', 'h': {'id': '220', 'title': 'Brighton', 'short_title': 'BRI'}, 'a': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 13:00:00'}, {'id': '21955', 'isResult': False, 'side': 'a', 'h': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'a': {'id': '220', 'title': 'Brighton', 'short_title': 'BRI'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 11:30:00'}, {'id': '21966', 'isResult': False, 'side': 'h', 'h': {'id': '220', 'title': 'Brighton', 'short_title': 'BRI'}, 'a': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21980', 'isResult': False, 'side': 'a', 'h': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'a': {'id': '220', 'title': 'Brighton', 'short_title': 'BRI

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B7', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Burnley"))

[{'id': '22289', 'isResult': False, 'side': 'a', 'h': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'a': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-08-19 14:00:00'}, {'id': '21944', 'isResult': False, 'side': 'a', 'h': {'id': '249', 'title': 'Nottingham Forest', 'short_title': 'NOT'}, 'a': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-18 18:45:00'}, {'id': '21950', 'isResult': False, 'side': 'h', 'h': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'a': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 19:00:00'}, {'id': '21958', 'isResult': False, 'side': 'a', 'h': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'a': {'id': '92', 'title': 'Burnley', 'short_title': '

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B8', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Chelsea"))

[{'id': '21942', 'isResult': False, 'side': 'a', 'h': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'a': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-17 13:00:00'}, {'id': '21947', 'isResult': False, 'side': 'h', 'h': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'}, 'a': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21964', 'isResult': False, 'side': 'a', 'h': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'a': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-02 19:00:00'}, {'id': '21967', 'isResult': False, 'side': 'a', 'h': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'a': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'}, 'goals': {'h':

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B9', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Crystal Palace"))

[{'id': '21948', 'isResult': False, 'side': 'h', 'h': {'id': '78', 'title': 'Crystal Palace', 'short_title': 'CRY'}, 'a': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21957', 'isResult': False, 'side': 'a', 'h': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'}, 'a': {'id': '78', 'title': 'Crystal Palace', 'short_title': 'CRY'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21968', 'isResult': False, 'side': 'h', 'h': {'id': '78', 'title': 'Crystal Palace', 'short_title': 'CRY'}, 'a': {'id': '249', 'title': 'Nottingham Forest', 'short_title': 'NOT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21981', 'isResult': False, 'side': 'a', 'h': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'a': {'id': '78', 'title': 'Crys

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B10', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Everton"))

[{'id': '21941', 'isResult': False, 'side': 'h', 'h': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'a': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-17 15:30:00'}, {'id': '21949', 'isResult': False, 'side': 'a', 'h': {'id': '244', 'title': 'Brentford', 'short_title': 'BRE'}, 'a': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 16:30:00'}, {'id': '21956', 'isResult': False, 'side': 'h', 'h': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'a': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21969', 'isResult': False, 'side': 'h', 'h': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'a': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'goals': {'h': N

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B11', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Fulham"))

[{'id': '21948', 'isResult': False, 'side': 'a', 'h': {'id': '78', 'title': 'Crystal Palace', 'short_title': 'CRY'}, 'a': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21964', 'isResult': False, 'side': 'h', 'h': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'a': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-02 19:00:00'}, {'id': '21970', 'isResult': False, 'side': 'h', 'h': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'a': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21984', 'isResult': False, 'side': 'a', 'h': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'a': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'go

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B12', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Liverpool"))

[{'id': '21953', 'isResult': False, 'side': 'h', 'h': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'a': {'id': '81', 'title': 'West Ham', 'short_title': 'WHU'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 13:00:00'}, {'id': '21962', 'isResult': False, 'side': 'a', 'h': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'a': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 16:30:00'}, {'id': '21966', 'isResult': False, 'side': 'a', 'h': {'id': '220', 'title': 'Brighton', 'short_title': 'BRI'}, 'a': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21979', 'isResult': False, 'side': 'h', 'h': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'a': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'goals': 

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B13', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Luton"))

[{'id': '22289', 'isResult': False, 'side': 'h', 'h': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'a': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-08-19 14:00:00'}, {'id': '21945', 'isResult': False, 'side': 'h', 'h': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'a': {'id': '229', 'title': 'Wolverhampton Wanderers', 'short_title': 'WOL'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21956', 'isResult': False, 'side': 'a', 'h': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'}, 'a': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21971', 'isResult': False, 'side': 'h', 'h': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'a': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'goals'

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B15', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Manchester United"))

[{'id': '21950', 'isResult': False, 'side': 'a', 'h': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'a': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 19:00:00'}, {'id': '21957', 'isResult': False, 'side': 'h', 'h': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'}, 'a': {'id': '78', 'title': 'Crystal Palace', 'short_title': 'CRY'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21972', 'isResult': False, 'side': 'h', 'h': {'id': '89', 'title': 'Manchester United', 'short_title': 'MUN'}, 'a': {'id': '244', 'title': 'Brentford', 'short_title': 'BRE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21983', 'isResult': False, 'side': 'a', 'h': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'a': {'id': '89', 'title': 'Manch

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B16', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Newcastle United"))

[{'id': '21954', 'isResult': False, 'side': 'a', 'h': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'a': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 15:30:00'}, {'id': '21958', 'isResult': False, 'side': 'h', 'h': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'a': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21973', 'isResult': False, 'side': 'a', 'h': {'id': '81', 'title': 'West Ham', 'short_title': 'WHU'}, 'a': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21981', 'isResult': False, 'side': 'h', 'h': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'a': {'id': '78', 'title': 'Crystal 

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B17', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Nottingham Forest"))

[{'id': '21944', 'isResult': False, 'side': 'h', 'h': {'id': '249', 'title': 'Nottingham Forest', 'short_title': 'NOT'}, 'a': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-18 18:45:00'}, {'id': '21946', 'isResult': False, 'side': 'a', 'h': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'a': {'id': '249', 'title': 'Nottingham Forest', 'short_title': 'NOT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21963', 'isResult': False, 'side': 'h', 'h': {'id': '249', 'title': 'Nottingham Forest', 'short_title': 'NOT'}, 'a': {'id': '244', 'title': 'Brentford', 'short_title': 'BRE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-01 13:00:00'}, {'id': '21968', 'isResult': False, 'side': 'a', 'h': {'id': '78', 'title': 'Crystal Palace', 'short_title': 'CRY'}, 'a': {'id': '249', 'title': 'Not

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B18', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Sheffield United"))

[{'id': '21954', 'isResult': False, 'side': 'h', 'h': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'a': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 15:30:00'}, {'id': '21960', 'isResult': False, 'side': 'a', 'h': {'id': '81', 'title': 'West Ham', 'short_title': 'WHU'}, 'a': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21970', 'isResult': False, 'side': 'a', 'h': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'a': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21983', 'isResult': False, 'side': 'h', 'h': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'a': {'id': '89', 'title': 'Manch

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B19', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Tottenham"))

[{'id': '21952', 'isResult': False, 'side': 'a', 'h': {'id': '83', 'title': 'Arsenal', 'short_title': 'ARS'}, 'a': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 13:00:00'}, {'id': '21962', 'isResult': False, 'side': 'h', 'h': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'a': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 16:30:00'}, {'id': '21971', 'isResult': False, 'side': 'a', 'h': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'a': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21984', 'isResult': False, 'side': 'h', 'h': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'}, 'a': {'id': '228', 'title': 'Fulham', 'short_title': 'FLH'}, 'goals': {'h'

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B20', [fixtures])

# Usage
asyncio.run(get_team_fixtures("West Ham"))

[{'id': '21953', 'isResult': False, 'side': 'a', 'h': {'id': '87', 'title': 'Liverpool', 'short_title': 'LIV'}, 'a': {'id': '81', 'title': 'West Ham', 'short_title': 'WHU'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-24 13:00:00'}, {'id': '21960', 'isResult': False, 'side': 'h', 'h': {'id': '81', 'title': 'West Ham', 'short_title': 'WHU'}, 'a': {'id': '238', 'title': 'Sheffield United', 'short_title': 'SHE'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21973', 'isResult': False, 'side': 'h', 'h': {'id': '81', 'title': 'West Ham', 'short_title': 'WHU'}, 'a': {'id': '86', 'title': 'Newcastle United', 'short_title': 'NEW'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21976', 'isResult': False, 'side': 'a', 'h': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'a': {'id': '81', 'title': 'West Ham', 'short_title': '

In [ ]:
async def get_team_fixtures(team_name):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        fixtures = await understat.get_team_fixtures(team_name, 2023)
        print(fixtures)
        next_5_opponents = []
        for fixture in fixtures[:5]:
          opponent = fixture['a']['title'] if fixture['side'] == 'h' else fixture['h']['title']
          match_date = fixture['datetime']
          next_5_opponents.append({'opponent': opponent, 'match_date': match_date})
        print(next_5_opponents)

        # Extract data into lists
        fixtures = [x['opponent'] for x in next_5_opponents]

        # Update Google Sheet
        fixtures_data_sheet.update('B21', [fixtures])

# Usage
asyncio.run(get_team_fixtures("Wolverhampton Wanderers"))

[{'id': '21945', 'isResult': False, 'side': 'a', 'h': {'id': '256', 'title': 'Luton', 'short_title': 'LUT'}, 'a': {'id': '229', 'title': 'Wolverhampton Wanderers', 'short_title': 'WOL'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-23 14:00:00'}, {'id': '21961', 'isResult': False, 'side': 'h', 'h': {'id': '229', 'title': 'Wolverhampton Wanderers', 'short_title': 'WOL'}, 'a': {'id': '88', 'title': 'Manchester City', 'short_title': 'MCI'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-09-30 14:00:00'}, {'id': '21974', 'isResult': False, 'side': 'h', 'h': {'id': '229', 'title': 'Wolverhampton Wanderers', 'short_title': 'WOL'}, 'a': {'id': '71', 'title': 'Aston Villa', 'short_title': 'AVL'}, 'goals': {'h': None, 'a': None}, 'xG': {'h': None, 'a': None}, 'datetime': '2023-10-07 14:00:00'}, {'id': '21975', 'isResult': False, 'side': 'a', 'h': {'id': '73', 'title': 'Bournemouth', 'short_title': 'BOU'}, 'a': {'id': '229'